In [1]:
import pandas as pd

# Load stale data
stale_data = pd.read_csv('dataset.csv')

# Load real-time data
real_time_data = pd.read_csv('node1_data.csv')

# Combine the two datasets
data = pd.concat([stale_data, real_time_data], ignore_index=True)

In [2]:
non_numeric_columns = data.select_dtypes(exclude=['number']).columns
print("Non-numeric columns:", non_numeric_columns)

# Inspect unique values in these columns
for column in non_numeric_columns:
    print(f"Unique values in {column}:", data[column].unique())

Non-numeric columns: Index(['car_id', 'model', 'node', 'charging'], dtype='object')
Unique values in car_id: ['ABC789' 'ABC456' 'ABCD123' 'ABCD456' 'WQAS321' 'WQAS543' 'ZASW1234'
 'Wien1' 'Wien2' 'Wien3' 'Wien4' 'Wien5' 'Wien6' 'Wien7' 'Wien8' 'Wien9'
 'Wien10' 'ABC123' 'ABD165' 'ABF185' 'ABE456']
Unique values in model: ['Model W' 'Model G' 'Model F' 'Model E' 'Model Q' 'Model M' 'Model S']
Unique values in node: ['Node 1']
Unique values in charging: [False]


In [3]:
data.drop(columns=['car_id'], inplace=True)

# One-hot encode potential categorical columns
data = pd.get_dummies(data, drop_first=True)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming the last column is the target variable
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=10, validation_split=0.2)


2023-08-14 00:12:42.216545: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 00:12:42.317526: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-14 00:12:42.320031: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-14 00:12:42.320040: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

Epoch 1/50


2023-08-14 00:12:43.304434: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-08-14 00:12:43.304450: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-14 00:12:43.304466: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debian): /proc/driver/nvidia/version does not exist
2023-08-14 00:12:43.304655: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1902/1902 [==============================] - 2s 795us/step - loss: 0.0411 - accuracy: 0.9881 - val_loss: 6.9407e-04 - val_accuracy: 1.0000
Epoch 2/50
1902/1902 [==============================] - 1s 731us/step - loss: 2.3363e-04 - accuracy: 1.0000 - val_loss: 8.1668e-05 - val_accuracy: 1.0000
Epoch 3/50
1902/1902 [==============================] - 1s 731us/step - loss: 3.8781e-05 - accuracy: 1.0000 - val_loss: 1.9740e-05 - val_accuracy: 1.0000
Epoch 4/50
1902/1902 [==============================] - 1s 787us/step - loss: 1.0489e-05 - accuracy: 1.0000 - val_loss: 5.9030e-06 - val_accuracy: 1.0000
Epoch 5/50
1902/1902 [==============================] - 1s 749us/step - loss: 3.2246e-06 - accuracy: 1.0000 - val_loss: 1.8620e-06 - val_accuracy: 1.0000
Epoch 6/50
1902/1902 [==============================] - 1s 743us/step - loss: 1.0437e-06 - accuracy: 1.0000 - val_loss: 6.2153e-07 - val_accuracy: 1.0000
Epoch 7/50
1902/1902 [==============================] - 1s 764us/step - loss: 3.4630e-07 - 

In [6]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy on test data: {accuracy:.2f}")

186/186 [==============================] - 0s 686us/step - loss: 1.3495e-09 - accuracy: 1.0000
Accuracy on test data: 1.00
